In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import cv2
import os, glob
import tensorflow as tf
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, AveragePooling2D,GlobalAveragePooling2D,Input
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
data_dir = r"Dataset\oil_dataset\test"
oil=[]
for file in os.listdir(data_dir):
    oil+=[file]
print(oil)
print(len(oil))

In [ ]:
img_height,img_width=256,256
batch_size=20

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory("Dataset/oil_dataset/train",
                                                target_size=(img_height,img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                subset='training')
test_generator = train_datagen.flow_from_directory("Dataset/oil_dataset/test",
                                                target_size=(img_height,img_width),
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                )

In [ ]:
precision=tf.keras.metrics.Precision()
recall=tf.keras.metrics.Recall()
sensitivity=tf.keras.metrics.SensitivityAtSpecificity(0.1)
specificity=tf.keras.metrics.SpecificityAtSensitivity(0.1)

In [ ]:
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
base_model3 =tf.keras.applications.ResNet50(input_shape=(img_height,img_width, 3), include_top=False,
                          weights='imagenet')
model3 = Sequential()
model3.add(base_model3)
model3.add(GlobalAveragePooling2D())
model3.add(Dense(64, activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))
model3.add(Dense(2, activation='sigmoid'))
model3.summary()

In [ ]:
model3.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy',precision,recall,sensitivity,specificity])
hist3=model3.fit(train_generator,epochs=15,validation_data=test_generator)

In [ ]:
model3.save("models/ResNet.h5")

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(hist3.history['accuracy'], color='b', label="Training accuracy")
ax[0].plot(hist3.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(hist3.history['loss'], color='b', label="Training loss")
ax[1].plot(hist3.history['val_loss'], color='r', label="validation loss",axes =ax[1])
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(hist3.history['precision'], color='b', label="Training precision")
ax[0].plot(hist3.history['val_precision'], color='r',label="Validation precision")
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(hist3.history['loss'], color='b', label="Training loss")
ax[1].plot(hist3.history['val_loss'], color='r', label="validation loss",axes =ax[1])
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(hist3.history['recall'], color='b', label="Training recall")
ax[0].plot(hist3.history['val_recall'], color='r',label="Validation recall")
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(hist3.history['loss'], color='b', label="Training loss")
ax[1].plot(hist3.history['val_loss'], color='r', label="validation loss",axes =ax[1])
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(hist3.history['sensitivity_at_specificity'], color='b', label="Training sensitivity")
ax[0].plot(hist3.history['val_sensitivity_at_specificity'], color='r',label="Validation sensitivity")
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(hist3.history['loss'], color='b', label="Training loss")
ax[1].plot(hist3.history['val_loss'], color='r', label="validation loss",axes =ax[1])
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(hist3.history['specificity_at_sensitivity'], color='b', label="Training specificity")
ax[0].plot(hist3.history['val_specificity_at_sensitivity'], color='r',label="Validation specificity")
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(hist3.history['loss'], color='b', label="Training loss")
ax[1].plot(hist3.history['val_loss'], color='r', label="validation loss",axes =ax[1])
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
filenames = test_generator.filenames
nb_samples = len(filenames)
model3_predict = model3.predict_generator(test_generator,steps = nb_samples, verbose=2)

In [ ]:
y_true = test_generator.classes
y_pred = model3_predict.argmax(axis=1)
plot_confusion_matrix(y_true, y_pred, classes=['Non Oil Spill', 'Oil Spill'],
                      title='Confusion matrix')